Imports

In [254]:

import pandas as pd
import numpy as np

import unidecode
import re
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LinearRegression
import sklearn.metrics as sm
from sklearn.metrics import r2_score

#football_data.to_csv('cleaned_football_data.csv', index=False)

Read data

In [255]:
football_data = pd.read_csv('data/futebol.csv')


In [256]:
football_data.shape

(1564, 7)

In [257]:
football_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1564 entries, 0 to 1563
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Data          1564 non-null   object 
 1   Jogo          1564 non-null   object 
 2   Minutos       1473 non-null   object 
 3   Investimento  1564 non-null   int64  
 4   Odd           1564 non-null   float64
 5   Ganho         1564 non-null   float64
 6   Resultado     1564 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 85.7+ KB


Translate columms to english

In [258]:
football_data.rename(columns={
    'Data': 'Date',
    'Jogo': 'Match',
    'Minutos': 'Bet Type',
    'Investimento': 'Bet Amount',
    'Odd': 'Odds',
    'Ganho': 'Profit',
    'Resultado': 'Win/Loss'
}, inplace=True)

In [259]:
football_data.sample(10)

,Date,Match,Bet Type,Bet Amount,Odds,Profit,Win/Loss
86,2022-07-11,Colon x Velez,00 ao 10,10,1.57,5.7,Green
836,2022-08-31,Velez x Flamengo,60 aos 70,10,1.44,4.4,Green
1229,2022-10-07,Lyon x Toulouse,10 a 20,10,1.61,6.1,Green
879,2022-09-03,Gualaceo x Barcelona,20 a 30,10,1.61,6.1,Green
534,2022-08-13,Goiás x Avaí,20 ao 30,10,1.50,5.0,Green
424,2022-08-06,Rostock x Biefield,NaN,10,1.57,5.7,Green
823,2022-08-30,CAP x Palmeiras,10 ao 20,10,1.50,5.0,Green
355,2022-08-01,DC x Orlando,00 a 10,40,1.50,20.0,Red
937,2022-09-08,Sturm Graz x Midyjylland,20 a 30,10,1.61,6.1,Green
1213,2022-10-05,Flamengo x Inter,20 a 30,10,1.50,5.0,Green


Checking for dublicates

In [260]:
football_data.duplicated().sum()

22

In [261]:
#dropping duplicates
football_data.drop_duplicates(inplace=True)

Check for missing values

In [262]:
football_data.isnull().sum()

Date           0
Match          0
Bet Type      88
Bet Amount     0
Odds           0
Profit         0
Win/Loss       0
dtype: int64

In [263]:
football_data['Bet Type'].unique()

array(['60 ao 70', '10 ao 20', '20 ao 30', 'Gols', '00 ao 10',
       'Escanteio', 'Vencedor', '50 ao 60', '00 a 10', '30 ao 40', nan,
       'Cartões', '70 ao 80', 'Asiáticos', '00 a 10 ', 'Multipla',
       'aos 30', '20 aos 30', '10 aos 20', '60 aos 70', '20 a 30',
       '10 a 20', '60 a 70', '80 a 90', '70 a 80', '30 a 40', '50 a 60',
       'Múltipla', '20 a 30 ', '2 marcam', 'gols', 'Mult', 'Pedro',
       '60 a 70 ', 'Gols duplo'], dtype=object)

replaced null with zero

In [264]:
football_data.fillna(0, inplace=True)

reduce the trailing zeros in profit

In [265]:
football_data['Profit'] = football_data['Profit'].round(2)

In [266]:
football_data['Bet Type'].value_counts()

Bet Type
00 a 10       257
20 ao 30      193
60 ao 70      174
Gols          148
10 ao 20      147
20 a 30       125
10 a 20       103
60 a 70        92
0              88
00 ao 10       85
30 a 40        22
30 ao 40       17
70 a 80        15
50 a 60        11
50 ao 60       10
70 ao 80       10
Asiáticos       8
80 a 90         8
20 a 30         5
20 aos 30       4
Multipla        4
aos 30          2
2 marcam        2
60 aos 70       1
10 aos 20       1
00 a 10         1
Cartões         1
Vencedor        1
Múltipla        1
Escanteio       1
gols            1
Mult            1
Pedro           1
60 a 70         1
Gols duplo      1
Name: count, dtype: int64

Normalize the data lowercased, stripped, and accents removed


In [267]:
print("Unique Bet Type:", football_data['Bet Type'].nunique())


Unique Bet Type: 35


In [268]:
def normalize_bet_type(value):
    if pd.isna(value):
        return value
    return unidecode.unidecode(str(value).strip().lower())

football_data['Bet Type'] = football_data['Bet Type'].apply(normalize_bet_type)

Standardize the time ranges from "10 ao 20" or 10 a 20 to 10-20

In [269]:
def standardize_time_ranges(value):
    if isinstance(value, str):
        return (
            value.replace(" a ", "-")
                 .replace(" ao ", "-")
                 .replace(" aos ", "-")
                 .strip()
        )
    return value
football_data['Bet Type'] = football_data['Bet Type'].apply(standardize_time_ranges)

In [270]:
football_data['Bet Type'].value_counts()

Bet Type
00-10         343
20-30         327
60-70         268
10-20         251
gols          149
0              88
30-40          39
70-80          25
50-60          21
asiaticos       8
80-90           8
multipla        5
2 marcam        2
aos 30          2
pedro           1
mult            1
cartoes         1
vencedor        1
escanteio       1
gols duplo      1
Name: count, dtype: int64

In [271]:
#remove damaged data
football_data = football_data[football_data['Bet Type'] != 'Pedro']

Group rare bet types into "Others"

In [272]:
threshold = 10

value_counts = football_data['Bet Type'].value_counts()

rare_types = value_counts[value_counts < threshold].index

def group_rare_bet_types(bet_type):
    if bet_type in rare_types:
        return 'Other'
    return bet_type

football_data['Bet Type'] = football_data['Bet Type'].apply(group_rare_bet_types)

In [273]:
football_data['Bet Type'].value_counts()

Bet Type
00-10    343
20-30    327
60-70    268
10-20    251
gols     149
0         88
30-40     39
Other     31
70-80     25
50-60     21
Name: count, dtype: int64

In [274]:
print("Unique Bet Type:", football_data['Bet Type'].nunique())
#reduced from 35 to 10

Unique Bet Type: 10


In [275]:
football_data.dtypes

Date           object
Match          object
Bet Type       object
Bet Amount      int64
Odds          float64
Profit        float64
Win/Loss       object
dtype: object

splits the column "Match" to be "Home Team" and "Away Team". 

Checked sample data with https://www.football-data.org/coverage that it matches. Home and Away team

In [276]:
#remove known bad data
football_data = football_data[football_data['Match'] != 'Múltipla 4']

# Split the 'Match' column into 'Home Team' and 'Away Team'
football_data[['Home Team', 'Away Team']] = football_data['Match'].str.split(' x ', expand=True)



#drop match column
football_data = football_data.drop(columns=['Match'])

new_order = ['Date', 'Home Team', 'Away Team', 'Bet Type', 'Bet Amount', 'Odds', 'Profit', 'Win/Loss']

#Reorder the columns
football_data = football_data[new_order]

In [277]:
football_data[football_data['Away Team'].isnull()]

,Date,Home Team,Away Team,Bet Type,Bet Amount,Odds,Profit,Win/Loss
364,2022-08-02,Colon X Indepediente,None,10-20,10,1.53,5.3,Green
655,2022-08-21,Leeds X Chelsea,None,20-30,10,1.50,5.0,Green
944,2022-09-08,SP X Atl Go,None,10-20,10,1.44,4.4,Green


manuel fix for the damaged data

In [278]:

football_data['Home Team'] = football_data['Home Team'].str.lower().str.strip()

manual_fix_map = {
    'colon x indepediente': ['Colon', 'Indepediente'],
    'leeds x chelsea': ['Leeds', 'Chelsea'],
    'sp x atl go': ['SP', 'Atl Go']
}

for bad_value, (home_fixed, away_fixed) in manual_fix_map.items():
    mask = (football_data['Home Team'] == bad_value) & (football_data['Away Team'].isnull())
    football_data.loc[mask, 'Home Team'] = home_fixed
    football_data.loc[mask, 'Away Team'] = away_fixed

In [279]:
football_data[football_data['Away Team'].isnull()]

,Date,Home Team,Away Team,Bet Type,Bet Amount,Odds,Profit,Win/Loss


Standardize the team names

In [280]:
football_data['Home Team'].unique()

array(['inglaterra', 'ayacucho', 'lara', 'guairena', 'cordoba',
       'belgrano', 'mar', 'cruzeiro', 'col', 'brusque', 'huachipate',
       'braga', 'colorado', 'dallas', 'patchuca', 'galaxy', 'morelia',
       'boca', 'libertad', 'inter', 'sucre', 'hailstorm', 'ready',
       'argentinos', 'colon', 'ceara', 'palmeiras', 'detroit', 'mineros',
       'river', 'melgar', 'union', 'lanus', 'stalbaek', 'asane',
       'patriotas', 'selangor', 'aksur', 'forest', 'chicago', 'vila',
       'celaya', 'rosário', 'dynamo', 'portim', 'brighton', 'nyc',
       'lorenzo', 'atlanta', 'goias', 'platense', 'petrolero',
       'millionarios', 'houston', 'gualaceo', 'unido', 'pepo', 'pif',
       'rosemberg', 'kalmar', 'jorge w.', 'tolima', 'velez', 'ilves',
       'defensa', 'bagre', 'varnamo', 'guabira', 'turan', 'din', 'talaea',
       'el zamalek', 'alianza', 'banfield', 'austin', 'cimarrones',
       'cortuba', 'dc united', 'breidablik', 'operario', 'resistencia',
       'huracan', 'estudiantes', '

In [281]:
print("Unique home teams:", football_data['Home Team'].nunique())
print("Unique away teams:", football_data['Away Team'].nunique())


Unique home teams: 784
Unique away teams: 885


In [282]:
football_data.isnull().sum()

Date          0
Home Team     0
Away Team     0
Bet Type      0
Bet Amount    0
Odds          0
Profit        0
Win/Loss      0
dtype: int64

In [283]:
#normalize team names
football_data['Home Team'] = football_data['Home Team'].str.lower().str.strip()
football_data['Away Team'] = football_data['Away Team'].str.lower().str.strip()



In [284]:
team_name_map = {
    # Fix accents and formatting
    'rosário': 'rosario',
    'atlético mg': 'atl mg',
    'atlético fênix': 'atl fênix',
    'atlético madrid': 'atletico madrid',
    'universitário': 'universitario',
    'ceará': 'ceara',
    'cuiabá': 'cuiaba',
    'san lorenzo res': 'san lorenzo',
    'banfield res': 'banfield',
    'talleres res': 'talleres',
    '  gremio': 'gremio',
    'croácia': 'croatia',
    'países baixos': 'netherlands',
    'santa fé': 'santa fe',
    'atl fenix': 'atl fênix',
    'atlético go': 'atl go',
    'indepediente': 'independiente',
    'gales': 'wales',
    'del valle': 'ind. del valle',
    
    # Consolidate naming variations
    'ny rb': 'ny red bulls',
    'nyc': 'ny city',
    'ny': 'ny city',
    'charllote': 'charlotte',
    'ny red bulls': 'ny red bulls',  # unify spelling
    
    # Translate or unify other known aliases
    'zurique': 'zurich',
    'guilhermo': 'guillermo',
    'vila': 'vila nova',
    'croacia': 'croatia',
    'dinamarca': 'denmark',
    'país de gales': 'wales',
    'paises baixos': 'netherlands',
    'espanha': 'spain',
    'alemanha': 'germany',
    'frança': 'france',
    'islandia': 'iceland',
    'nikobing': 'nykobing',
}

football_data['Home Team'] = football_data['Home Team'].replace(team_name_map)
football_data['Away Team'] = football_data['Away Team'].replace(team_name_map)


In [285]:
#remove damaged data
bad_teams = ['A', 'B', 'FF', 'PK', 'Múltipla 4']
football_data = football_data[
    ~football_data['Home Team'].isin(bad_teams) &
    ~football_data['Away Team'].isin(bad_teams)
]

In [286]:
print("Unique home teams:", football_data['Home Team'].nunique())
print("Unique away teams:", football_data['Away Team'].nunique())


Unique home teams: 770
Unique away teams: 792


In [287]:
football_data.sample(10)

,Date,Home Team,Away Team,Bet Type,Bet Amount,Odds,Profit,Win/Loss
1219,2022-10-06,lanus,arg jrs,20-30,10,1.66,6.6,Green
137,2022-07-16,ham-kam,bodo,20-30,10,1.44,4.4,Green
1407,2022-10-19,elche,real madrid,60-70,10,1.57,5.7,Green
806,2022-08-29,start,grorud,20-30,40,1.50,20.0,Red
1461,2022-10-22,chelsea,man united,gols,2,1.50,1.0,Red
147,2022-07-16,flamengo,coritiba,00-10,10,1.53,5.3,Green
973,2022-09-10,ceara,santos,00-10,10,1.53,5.3,Green
1271,2022-10-10,audax,u. la calera,30-40,10,1.61,6.1,Green
1259,2022-10-09,boa vista,marítimo,00-10,10,1.57,5.7,Green
1449,2022-10-22,man city,brighton,gols,10,3.25,22.5,Green


Label encoding

In [288]:
football_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1541 entries, 0 to 1563
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Date        1541 non-null   object 
 1   Home Team   1541 non-null   object 
 2   Away Team   1541 non-null   object 
 3   Bet Type    1541 non-null   object 
 4   Bet Amount  1541 non-null   int64  
 5   Odds        1541 non-null   float64
 6   Profit      1541 non-null   float64
 7   Win/Loss    1541 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 108.4+ KB


convert Win/Loss to binary

In [289]:
football_data['Win/Loss'] = football_data['Win/Loss'].map({'Green': 1, 'Red': 0})

change Bet type to One-hot encoding

In [290]:
#football_data = pd.get_dummies(football_data, columns=['Bet Type'], prefix='Bet')

In [291]:
football_data.head()

,Date,Home Team,Away Team,Bet Type,Bet Amount,Odds,Profit,Win/Loss
0,2022-07-01,inglaterra,israel,60-70,10,1.57,5.7,1
1,2022-07-01,ayacucho,cantolao,10-20,10,1.50,5.0,1
2,2022-07-01,ayacucho,cantolao,20-30,10,1.50,5.0,1
3,2022-07-01,lara,"ucv ac. 0,5 gols",gols,10,1.61,6.1,0
4,2022-07-01,guairena,asuncion,20-30,10,1.57,5.7,1


label encode home and away team and make sure a team in Home will have the same ID in Away

In [292]:
##all_teams = pd.concat([football_data['Home Team'], football_data['Away Team']]).unique()

# Fit one LabelEncoder on all teams
##team_encoder = LabelEncoder()
##team_encoder.fit(all_teams)

# Apply the same encoder to both columns
##football_data['Home Team'] = team_encoder.transform(football_data['Home Team'])
##football_data['Away Team'] = team_encoder.transform(football_data['Away Team'])

In [293]:
#removes whitespace from column names and replaces spaces with underscores
football_data.columns = [col.strip().replace(' ', '_') for col in football_data.columns]
football_data.to_csv('cleaned_football_data.csv', index=False)

In [294]:
football_data.sample(10)

,Date,Home_Team,Away_Team,Bet_Type,Bet_Amount,Odds,Profit,Win/Loss
1250,2022-10-08,corinthians,athletico pr,00-10,10,1.53,5.3,1
375,2022-08-03,maryland,cal,20-30,10,1.50,5.0,0
403,2022-08-05,randers,horsens,0,10,1.57,5.7,1
1239,2022-10-08,hamburgo,kaiserslautern,00-10,10,1.53,5.3,1
1451,2022-10-22,man city,brighton,gols,10,3.40,24.0,1
719,2022-08-24,flu,corinthians,20-30,10,1.50,5.0,1
1169,2022-10-01,brusque,criciuma,60-70,40,1.50,20.0,0
1347,2022-10-13,junior,u. magdalena,20-30,10,1.66,6.6,1
87,2022-07-11,colon,velez,10-20,10,1.50,5.0,1
1080,2022-09-19,feirense,olivereinse,30-40,10,1.50,5.0,1
